In [50]:
import pandas as pd 

df = pd.read_csv('ecommerce_dataset.csv')
print(df.shape)

(50425, 2)


In [51]:
df.head()

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [52]:
df.columns = df.columns.str.strip() #as I have manually added the columns of category and description there might be a whitespace so lets strip the shape :

df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [53]:
df.isna().sum()

category       0
description    1
dtype: int64

In [54]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [55]:
#we can see there are three different categories lets now just rename the thrid one without anyspaces:

df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace= True )


In [56]:
df.category.value_counts()

category
Household               19313
Books                   11820
Electronics             10621
Clothing_Accessories     8670
Name: count, dtype: int64

In [57]:
#Fastext is desined as it takes _label__ as fist in the category section, lets prefix the label for that :

df['category'] = "__label__" + df['category'].astype(str)
df.head()


,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [58]:
#now lets merge the category adn decription together :

df['category_description'] = df['category'] + " " + df['description']
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [59]:
df['category_description'][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

As you can see that their is some special char so lets first remove all that 

In [60]:
# we can do that using the regular expression: 

import re

text = " Rohit's | LoveS playing : Cricket(Bating/Bowling, for India) | ? ; Yes/nO"

def preprocess(text):
    text = re.sub(r'[^\w\s\']', " ", text)
    text = re.sub(r' +', " ", text)
    return text.strip().lower()


In [61]:
preprocess(" Rohit's | LoveS playing : Cricket(Bating/Bowling, for India) | ? ; Yes/nO")

"rohit's loves playing cricket bating bowling for india yes no"

In [62]:
df['category_description']  = df['category_description'].map(preprocess)

In [63]:
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [64]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)

In [65]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [66]:
test.head()

,category,description,category_description
3957,__label__Household,Ajanta Plastic Wall Clock (20.5 cm x 20.5 cm x...,__label__household ajanta plastic wall clock 2...
2566,__label__Household,MBTC Three Seater Airport Visitor Waiting Chai...,__label__household mbtc three seater airport v...
47672,__label__Electronics,CEUTA Wireless Car Bluetooth 2.5A Dual Usb Tur...,__label__electronics ceuta wireless car blueto...
36761,__label__Clothing_Accessories,Wet Pet Dog Running Jogging Lead Elasticated F...,__label__clothing_accessories wet pet dog runn...
35485,__label__Clothing_Accessories,Analogue Men's Blue Dial Watch and Wayfarer Su...,__label__clothing_accessories analogue men's b...


In [67]:
#As we need to create a dataset with only category_description:

train.to_csv("ecommerce.train", columns=['category_description'], index=False, header=False)
test.to_csv('ecommerce.test', columns=['category_description'], index=False, header=False)


In [79]:
import fasttext

modelft = fasttext.train_supervised(input="ecommerce.train")
modelft.test('ecommerce.test')

Read 4M words
Number of words:  79276
Number of labels: 4
Progress: 100.0% words/sec/thread: 3261689 lr:  0.000000 avg.loss:  0.174165 ETA:   0h 0m 0s


(10085, 0.9656916212196331, 0.9656916212196331)

In [80]:
modelft.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [84]:
modelft.get_nearest_neighbors("cotton")

[(0.9944185614585876, 'piece'),
 (0.9941878318786621, 'cold'),
 (0.9935899376869202, 'rose'),
 (0.991212785243988, 'square'),
 (0.9904393553733826, '335'),
 (0.9900165796279907, 'summer'),
 (0.9893838167190552, 'little'),
 (0.9852024912834167, 'royal'),
 (0.9846295118331909, 'difference'),
 (0.9843281507492065, 'designs')]